In [ ]:
import pandas as pd 
order_log=pd.read_csv("order_log_clean.csv") 
menu=pd.read_csv("menu_clean.csv") 





#Task 2: Truy vấn và thống kê mô tả 

Thống kê số món bán theo từng khung giờ

In [ ]:

import pandas as pd

order_log["order_time"] = pd.to_datetime(order_log["order_time"], errors="coerce") #chuyển sang datime thật
order_log = order_log.dropna(subset=["order_time"])  # xóa các dòng ko có thời gian đặt hàng 
# Tạo cột hour từ order_time 
order_log["hour"] = order_log["order_time"].dt.hour 
# .dt là định dạng datime truy cập thuộc tính thời gian , .hour : lấy giờ trong ngày (0-23)

#  Tạo dtf 0–23 giờ để đảm bảo đủ khung giờ
hours = pd.DataFrame({"hour": range(24)}) # Tên cột "hour" giá trị chạy từ 0-23

#  Thống kê tổng số món bán theo từng giờ
qty_hour = order_log.groupby("hour")["qty"].sum().reset_index(name="total_sold") #Gom dòng dữ liệu theo từng giờ, sau đó tính tổng số món bán qty trong mỗi giờ  
qty_hour = hours.merge(qty_hour, on="hour", how="left").fillna(0) # .finall(0): các giờ không có đơn hàng NaN đổi thành 0 món bán
qty_hour["total_sold"] = qty_hour["total_sold"].astype(int) # Chuyển cột total_sold từ float sang int


#  In kết quả
print(qty_hour)


    hour  total_sold
0      0           1
1      1           4
2      2           7
3      3           1
4      4           7
5      5           6
6      6          14
7      7           3
8      8           3
9      9           2
10    10           7
11    11           5
12    12           3
13    13           4
14    14           5
15    15           4
16    16           3
17    17           8
18    18           2
19    19           1
20    20           2
21    21           4
22    22           4
23    23           4


Tìm số món bán chạy nhất

In [7]:
best_selling = order_log.groupby("product_id")["qty"].sum().reset_index(name="total_bestselling") 
# Gom các dòng dữ liệu product_id lại với nhau sau đó tính tổng số lượng qty, sau đó đặt tên tổng số lượng đó
best_selling = best_selling.sort_values(by="total_bestselling", ascending=False) 
# Lấy id 5 món bán chạy nhất bằng cách sắp xếp giảm dần total_bestselling
print(" Sau đây là danh sách những món bán chạy nhất")
print(best_selling)

 Sau đây là danh sách những món bán chạy nhất
  product_id  total_bestselling
1       CF02               25.0
3       CF04               24.0
2       CF03               18.0
0       CF01               13.0
4       CF05               12.0
5       CF06               12.0


Tính doanh thu theo từng món 

In [8]:
# Merge order_log với menu để lấy giá 
df_revenue = order_log.merge(menu[["product_id","price"]], on="product_id", how="left")
df_revenue["price"] = df_revenue["price"].astype(float) # Chuyển đổi giá thành số thực
df_revenue["revenue"] = df_revenue["qty"] * df_revenue["price"] # revenue= qty * price 

# Tính tổng doanh thu theo từng món
revenue_product = df_revenue.groupby("product_id")["revenue"].sum().reset_index() #Tính tổng doanh thu 

revenue_product = revenue_product.sort_values(by="revenue", ascending=False) # Sắp xếp doanh thu của từng món theo thứ tự giảm dần
print(" Danh sách tổng danh thu của từng món")
print(revenue_product)


 Danh sách tổng danh thu của từng món
  product_id    revenue
3       CF04  1080000.0
1       CF02   875000.0
2       CF03   630000.0
0       CF01   585000.0
5       CF06   420000.0
4       CF05      360.0


Tìm đơn hàng bị thiếu trong thông tin món 

In [9]:
missing_product = order_log[~order_log["product_id"].isin(menu["product_id"])]
#Lọc ra các đơn hàng có product_id không tồn tại trong menu ( trong đó ~ là phủ định, isin: có trong product_id của menu hay không)
print("Các đơn hàng thiếu thông tin món")
print(missing_product)

Các đơn hàng thiếu thông tin món
Empty DataFrame
Columns: [order_id, product_id, order_time, qty, hour]
Index: []


Thống kê số đơn theo ngày 

In [10]:
order_log["date"] = order_log["order_time"].dt.date # lấy ngày trong ordertime bằng định dạng .dt.date
orders_day = order_log.groupby("date")["order_id"].count().reset_index(name="total_orders") # .count() đếm số mã đơn hàng order_id trong một ngày
print("Tổng số đơn hàng theo ngày")
print(orders_day)

Tổng số đơn hàng theo ngày
         date  total_orders
0  2024-01-01            52
